In [ ]:
#Import PV-DER modules
from pvder.DER_components_single_phase import SolarPV_DER_SinglePhase
from pvder.DER_components_three_phase  import SolarPV_DER_ThreePhase
#from pvder.DER_components_three_phase_balanced  import SolarPV_DER_ThreePhaseBalanced
from pvder.grid_components import Grid
from pvder.dynamic_simulation import DynamicSimulation
from pvder.simulation_events import SimulationEvents
from pvder.simulation_utilities import SimulationUtilities,SimulationResults
from pvder import utility_functions

#Import Python modules
import logging
import numpy as np
import scipy as sci
import math
import cmath
import pandas


import matplotlib as matplotlib
import matplotlib.pyplot as plt
print('Scipy version:',sci.__version__)
print('Numpy version:',np.__version__)
print('Matplotlib version:',matplotlib.__version__)
#To show interactive plot in the notebook cell
%matplotlib inline    

In [ ]:
voltages = pandas.read_csv('results_powerfactory.csv')
voltages.head()
Va_t = np.array(voltages[['Vmag','Vangle']])
fgrid_t = np.array(voltages[['fgrid']])*60.0
print(Va_t.shape,fgrid_t.shape)
Va = cmath.rect(Va_t[0][0]*Grid.Vbase/2, math.radians(Va_t[0][1]))
Vb = utility_functions.Ub_calc(Va)
Vc = utility_functions.Uc_calc(Va)

Vrms = abs(Va)/math.sqrt(2)
ph= Va_t[0][1]
print('Vrms:{:.2f},Angle:{:.2f},fgrid:{:.2f}'.format(Vrms,ph,fgrid_t[0,0]))
print('Va:{:.2f},Vb:{:.2f},Vc:{:.2f}'.format(Va,Vb,Vc))
print('V0:',Va+Vb+Vc)

Vat = []
Vbt = []
Vct = []
fgridt = []
for i in range(len(Va_t)):
    Vat.append(cmath.rect(Va_t[i][0]*Grid.Vbase/2, math.radians(Va_t[i][1])))
    Vbt.append(utility_functions.Ub_calc(Vat[i]))
    Vct.append(utility_functions.Uc_calc(Vat[i]))
    fgridt.append(fgrid_t[i,0])    

In [ ]:
logging.debug('test')
SINGLE_PHASE = False
STEADY_STATE = True
UNBALANCED = False
events1 = SimulationEvents(verbosity = 'DEBUG')
if SINGLE_PHASE:
    PV_DER1 = SolarPV_DER_SinglePhase(events=events1,Sinverter_rated = 10.0e3,Vrms_rated = Vrms, #175
                                     gridVoltagePhaseA = Va,
                                     gridVoltagePhaseB = Vb,
                                     gridVoltagePhaseC = Vc,
                                     gridFrequency=2*math.pi*60.0,
                                     standAlone = False,STEADY_STATE_INITIALIZATION=STEADY_STATE,allow_unbalanced_m=UNBALANCED,
                                     verbosity = 'DEBUG')
    
else:
    PV_DER1 = SolarPV_DER_ThreePhase(events=events1,Sinverter_rated = 50.0e3,Vrms_rated = Vrms, #175
                                     gridVoltagePhaseA = Va,
                                     gridVoltagePhaseB = Vb,
                                     gridVoltagePhaseC = Vc,
                                     gridFrequency=2*math.pi*60.0,
                                     standAlone = False,STEADY_STATE_INITIALIZATION=STEADY_STATE,allow_unbalanced_m=UNBALANCED,
                                     verbosity = 'DEBUG')
      
        
    
sim1 = DynamicSimulation(PV_model=PV_DER1,events = events1,verbosity = 'INFO',solver_type='odeint',LOOP_MODE=True) #'odeint','ode-vode-bdf'
results1 = SimulationResults(simulation = sim1,PER_UNIT=True,verbosity = 'INFO')

In [ ]:
%%time
PV_DER1.MPPT_ENABLE=False
PV_DER1.RAMP_ENABLE = False
PV_DER1.VOLT_VAR_ENABLE = False
PV_DER1.LVRT_ENABLE = False
PV_DER1.HVRT_ENABLE = False
PV_DER1.LFRT_ENABLE = False
PV_DER1.LVRT_MOMENTARY_CESSATION = True
PV_DER1.DO_EXTRA_CALCULATIONS = False
PV_DER1.use_frequency_estimate=False
sim1.jacFlag = True
sim1.DEBUG_SIMULATION = False
sim1.DEBUG_VOLTAGES = True
sim1.DEBUG_CURRENTS = True
sim1.DEBUG_POWER = False
sim1.DEBUG_CONTROLLERS  = True
sim1.DEBUG_PLL = False
sim1.PER_UNIT = True
sim1.DEBUG_SOLVER  = False
sim1.tStop = 10.0
sim1.tInc = 1/120.# 0.001 #0.0125

sim1.playback_3phgrid(Vat,Vbt,Vct,fgridt)#fgridt,60.0
PV_DER1.validate_model()

PV_DER1.show_PV_DER_states('voltage')
PV_DER1.show_PV_DER_states('current')
PV_DER1.show_PV_DER_states('power')
PV_DER1.show_PV_DER_states('duty cycle')

In [ ]:
utility_functions.numpy_to_csv('P_PCC',sim1.S_PCC_t.real,'P_PCC')

In [ ]:
results1.PER_UNIT = False
results1.PLOT_TITLE = True
results1.font_size = 18
results1.plot_DER_simulation(plot_type='active_power_Ppv_Pac_PCC')#

In [ ]:
results1.plot_DER_simulation(plot_type='reactive_power')#_Q_PCC

In [ ]:
results1.plot_DER_simulation(plot_type='voltage_Vpcclv_all_phases') #voltage_Vpcclv

In [ ]:
results1.plot_DER_simulation(plot_type='voltage_Vdc')

In [ ]:
results1.plot_DER_simulation(plot_type='current')

In [ ]:
results1.plot_DER_simulation(plot_type='duty_cycle')

In [ ]:
results1.plot_DER_simulation(plot_type='frequency')

In [ ]:
results1.plot_DER_simulation(plot_type='pll')